### Inserting Data into PostgreSQL

#### Table created in PostgreSQL for this project is called "diabetic_ret"

In [2]:
!pip install psycopg2

     |████████████████████████████████| 380 kB 1.6 MB/s eta 0:00:01
  Created wheel for psycopg2: filename=psycopg2-2.8.5-cp37-cp37m-macosx_10_9_x86_64.whl size=136140 sha256=b3addd2ade63b934346ebfde62cf2bafdfdb16f67eec1232ecc38a7e892c9edd
  Stored in directory: /Users/sonalidasgupta/Library/Caches/pip/wheels/69/74/ec/7764aea035627654bf137f89a91405ac239c569ce10ae87137
Successfully built psycopg2


In [1]:
#Import packages

import csv
import pandas as pd

import pandas.io.sql as pd_sql

from collections import defaultdict

import os

import psycopg2 as pg
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
from psycopg2 import OperationalError, errorcodes, errors

from tqdm import tqdm

import pprint as pp

In [2]:
#Get file from local computer 

sd_sample = pd.read_csv('dr.csv')
sd_sample.head()

,id,_0,_1,_2,_3,_4,_5,_6,_7,_8,...,_10,_11,_12,_13,_14,_15,_16,_17,_18,Class
0,1,1,1,22,22,22,19,18,14,49.895756,...,5.270920,0.771761,0.018632,0.006864,0.003923,0.003923,0.486903,0.100025,1,0
1,2,1,1,24,24,22,18,16,13,57.709936,...,3.325423,0.234185,0.003903,0.003903,0.003903,0.003903,0.520908,0.144414,0,0
2,3,1,1,62,60,59,54,47,33,55.831441,...,12.687485,4.852282,1.393889,0.373252,0.041817,0.007744,0.530904,0.128548,0,1
3,4,1,1,55,53,53,50,43,31,40.467228,...,9.118901,3.079428,0.840261,0.272434,0.007653,0.001531,0.483284,0.114790,0,0
4,5,1,1,44,44,44,41,39,27,18.026254,...,0.410381,0.000000,0.000000,0.000000,0.000000,0.000000,0.475935,0.123572,0,1


#### Create Table Statement 

In [3]:
#Get types of variables 

sd_sample.dtypes

id         int64
_0         int64
_1         int64
_2         int64
_3         int64
_4         int64
_5         int64
_6         int64
_7         int64
_8       float64
_9       float64
_10      float64
_11      float64
_12      float64
_13      float64
_14      float64
_15      float64
_16      float64
_17      float64
_18        int64
Class      int64
dtype: object

In [4]:
sd_sample.columns

Index(['id', '_0', '_1', '_2', '_3', '_4', '_5', '_6', '_7', '_8', '_9', '_10',
       '_11', '_12', '_13', '_14', '_15', '_16', '_17', '_18', 'Class'],
      dtype='object')

In [5]:
#Create table statement:
def create_table_statement(df, table_name):
    """
        Helper function to make "CREATE TABLE" statement for psql
        
        :param df: a pandas DateFrame object
        :param table_name: name of new table in psql as string
        :return create_table: "CREATE TABLE" statement for psql as string
    """
    
    col_names = df.dtypes.index.values
    dtypes = df.dtypes.values

    #every name in psql is lowercase by default
    table_name = table_name.lower()

    #statement for creating the table in psql
    create_table = "CREATE TABLE IF NOT EXISTS %s (\n" %table_name  

    for idx, col_name in enumerate(col_names):
        #get the datatype for the current column
        dtype = dtypes[idx]

        #lowercase and remove spaces from column names
        col_name = col_name.lower().replace(' ', '_')
        #simply formatting so it prints more cleanly
        create_table += "\t"

        #assign column types and default values of null
        if dtype == 'int64':
            create_table = create_table + col_name + " INT DEFAULT NULL"

        elif dtype == 'float64':
            create_table = create_table + col_name + " DECIMAL DEFAULT NULL"

        #add a comma and start next line on 
        #another line for cleaner print format
        #if this is not the last line
        if idx != len(col_names) - 1:
            create_table += ", \n"

    #newline for cleaner print format and ; for execution
    create_table += "\n);"

    print (create_table)
    
    return create_table

In [9]:
create_table = create_table_statement(sd_sample, 'sd_data')

CREATE TABLE IF NOT EXISTS sd_data (
	id INT DEFAULT NULL, 
	_0 INT DEFAULT NULL, 
	_1 INT DEFAULT NULL, 
	_2 INT DEFAULT NULL, 
	_3 INT DEFAULT NULL, 
	_4 INT DEFAULT NULL, 
	_5 INT DEFAULT NULL, 
	_6 INT DEFAULT NULL, 
	_7 INT DEFAULT NULL, 
	_8 DECIMAL DEFAULT NULL, 
	_9 DECIMAL DEFAULT NULL, 
	_10 DECIMAL DEFAULT NULL, 
	_11 DECIMAL DEFAULT NULL, 
	_12 DECIMAL DEFAULT NULL, 
	_13 DECIMAL DEFAULT NULL, 
	_14 DECIMAL DEFAULT NULL, 
	_15 DECIMAL DEFAULT NULL, 
	_16 DECIMAL DEFAULT NULL, 
	_17 DECIMAL DEFAULT NULL, 
	_18 INT DEFAULT NULL, 
	class INT DEFAULT NULL
);


#### Connect to PostgreSQL 

In [6]:
connection_args = {'host': '52.52.92.166',
                   'user':'ubuntu', 
                   'password':'',
                   'port': 5432}

In [7]:
#Connect to psql 

connection = pg.connect(**connection_args)

#if you want to autocommit instead, comment out if not
connection.autocommit = True

cursor = connection.cursor()

#if you want to autocommit instead, comment out if not
connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

In [8]:
#Close and then connect using database I created 

connection.close()

connection_args['dbname'] = 'diabetic_ret'
connection = pg.connect(**connection_args)

cursor = connection.cursor()

#if you want to autocommit instead, comment out if not
connection.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

In [17]:
#create table in psql
cursor.execute(create_table)

In [18]:
#Insert table 

def insert_table_statement(df, table_name):
    """
        Function to make "INSERT INTO" statement for psql
        
        :param df: a pandas DateFrame object
        :param table_name: name of new table in psql as string
        :return insert_statement: "INSERT INTO" statement for psql as string
    """
    
    col_names = df.columns
    
    #statement for inserting values into table
    insert_statement = "INSERT INTO %s (" %table_name
    values = " VALUES ("
    insert_statement

    for idx, col_name in enumerate(col_names):

        #lowercase and remove spaces from column names
        #as we did before with the table
        col_name = col_name.lower().replace(' ', '_')

        #adding columns to insert statement
        #also formatting after every 4th column for cleaner print format
        if idx % 4 == 0 and idx != 0:
            insert_statement = insert_statement + '\n\t\t\t' + col_name
        else:
            insert_statement = insert_statement + col_name 

        #append a %s to values for every col_name
        #this is for substituting the values in later
        values += "%s"

        #exit loop if this the last column
        if idx == len(col_names) - 1:
            break

        insert_statement += ", "
        values += ", "

    insert_statement += ")"
    values += ");"

    insert_statement = insert_statement + '\n' + values

    print(insert_statement)
    
    return insert_statement

In [19]:
insert_table_statement(sd_sample, 'sd_data')

INSERT INTO sd_data (id, _0, _1, _2, 
			_3, _4, _5, _6, 
			_7, _8, _9, _10, 
			_11, _12, _13, _14, 
			_15, _16, _17, _18, 
			class)
 VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);


'INSERT INTO sd_data (id, _0, _1, _2, \n\t\t\t_3, _4, _5, _6, \n\t\t\t_7, _8, _9, _10, \n\t\t\t_11, _12, _13, _14, \n\t\t\t_15, _16, _17, _18, \n\t\t\tclass)\n VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);'

In [12]:
!ls

Datasets                                Project2-WebScraping_EDA.ipynb
Metis Project 2 Presentation_Sonali.pdf Python Challenges
Pair Programming Exercises              Quiz 1 answers.ipynb
Project 1-EDA .ipynb                    confusion_matrix_knn_mvp.png
Project 2-Linear Regression.ipynb       confusion_matrix_logit_mvp.png
Project 3-Classification .ipynb         dr_messidorfeatures.csv
Project 3-PostgreSQL.ipynb              sodas32.github.io


In [9]:
#ex: COPY AllstarFull FROM '/home/ubuntu/baseballdata/AllstarFull.csv' DELIMITER ',' CSV HEADER;
#COPY statement in psql as string
#copy_statement = "COPY " + 'sd_data' + " FROM " +  path + " DELIMITER ',' CSV HEADER;"
#/Users/sonalidasgupta/Metis /dr.csv

table_name = "sd_data"

f = open('dr.csv', encoding='utf-8', errors = 'ignore')

#copy data to psql table
cursor.copy_from(f, table_name)

#save data to psql table
connection.commit()

InvalidTextRepresentation: invalid input syntax for integer: "id,_0,_1,_2,_3,_4,_5,_6,_7,_8,_9,_10,_11,_12,_13,_14,_15,_16,_17,_18,Class"
CONTEXT:  COPY sd_data, line 1, column id: "id,_0,_1,_2,_3,_4,_5,_6,_7,_8,_9,_10,_11,_12,_13,_14,_15,_16,_17,_18,Class"


### After this, I ended up loading data into psql directly from my csv (dr.csv) in AWS 
##### Database name: diabetic_ret
##### Table name: sd_data